## Setup

In [1]:
import os
import requests
import json
from pathlib import Path

import sys
sys.path.append('../')
from modules.helpers import get_access_token, load_eoepca_state, test_cell, test_results

Load `eoepca state` environment

In [2]:
load_eoepca_state()

In [3]:
platform_domain = os.environ.get("INGRESS_HOST")
resource_registration_domain = f'{os.environ.get("HTTP_SCHEME")}://registration-api.{platform_domain}'

print(f"Resource Registration API: {resource_registration_domain}")

Resource Registration API: https://registration-api.deploybox.co.uk


## Validate Resource Registration API Endpoints

In [4]:
endpoints = [
    ("Landing Page", resource_registration_domain),
    ("Swagger UI", f"{resource_registration_domain}/openapi?f=html"),
    ("Conformance Declaration", f"{resource_registration_domain}/conformance"),
    ("Processes List", f"{resource_registration_domain}/processes"),
    ("Jobs Endpoint", f"{resource_registration_domain}/jobs")
]

for name, url in endpoints:
    response = requests.get(url)
    print(f"{name} ({url}): {response.status_code}")

Landing Page (https://registration-api.deploybox.co.uk): 200
Swagger UI (https://registration-api.deploybox.co.uk/openapi?f=html): 200
Conformance Declaration (https://registration-api.deploybox.co.uk/conformance): 200
Processes List (https://registration-api.deploybox.co.uk/processes): 200
Jobs Endpoint (https://registration-api.deploybox.co.uk/jobs): 200


## Inspect Available Processes

In [5]:
processes_url = f"{resource_registration_domain}/processes"
processes_response = requests.get(processes_url)
print(json.dumps(processes_response.json(), indent=2)[:500])

{
  "processes": [
    {
      "version": "0.1.0",
      "id": "deregister",
      "title": "Resource deregistration",
      "description": "Resource deregistration",
      "jobControlOptions": [
        "sync-execute",
        "async-execute"
      ],
      "keywords": [
        "resource",
        "deregistration"
      ],
      "links": [
        {
          "type": "text/html",
          "rel": "about",
          "title": "information",
          "href": "https://eoepca.readthedocs.io/projec


## Registering Remote STAC Collection

In [25]:
register_url = f"{resource_registration_domain}/processes/register/execution"

payload = {
    "inputs": {
        "type": "collection",
        "source": {"rel": "collection", "href": "https://raw.githubusercontent.com/james-hinton/temp-data-store/refs/heads/main/stac-collection.json"},
        "target": {"rel": "https://api.stacspec.org/v1.0.0/core", "href": f"https://resource-catalogue.{platform_domain}/stac"}
    }
}
response = requests.post(register_url, json=payload)
registration_result = response.json()
print(registration_result)

{'id': 'registrar', 'resource-and-data-catalogue-link': {'href': 'https://resource-catalogue.deploybox.co.uk/stac/collections/S2MSI2A', 'rel': 'item', 'type': 'application/geo+json'}}


In [12]:
# Confirm that the registration was successful
collection_href = registration_result["resource-and-data-catalogue-link"]['href']

response = requests.get(collection_href)
print(f"Collection successfully registered at {collection_href}")


Collection successfully registered at https://resource-catalogue.deploybox.co.uk/stac/collections/S2MSI2A


## Deregistering a STAC Collection

In [26]:
deregister_url = f"{resource_registration_domain}/processes/deregister/execution"
payload = {
    "inputs": {
        "id": "S2MSI2A",
        "target": {
            "rel": "https://api.stacspec.org/v1.0.0/core",
            "href": f"https://resource-catalogue.{platform_domain}/stac"
        },
        "rel": "collection"
        
    }
}
response = requests.post(deregister_url, json=payload)
print(response.json())

{'id': 'S2MSI2A'}


## Registering Sentinel Item

In [24]:
stac_item_url = 'https://raw.githubusercontent.com/james-hinton/temp-data-store/refs/heads/main/stac-item.json'
stac_item_json_data = requests.get(stac_item_url).json()

json_dict = {
    "inputs": {
      "type": "item",
      "source": "https://raw.githubusercontent.com/james-hinton/temp-data-store/refs/heads/main/stac-item.json",
      "target": f"https://resource-catalogue.{platform_domain}"
    }
}
response = requests.post(
    f"{resource_registration_domain}/processes/register/execution",
    json=json_dict
)
response.raise_for_status()
response


HTTPError: 400 Client Error: BAD REQUEST for url: https://registration-api.deploybox.co.uk/processes/register/execution

In [ ]:
# Deregister
# json_dict = {
#     "inputs": {
#       "id": "S2B_MSIL2A_20190910T095029_N0213_R079_T33UXP_20190910T124513.SAFE",
#       "target": f"https://resource-catalogue.{platform_domain}"
#     }
# }
# response = requests.post(
#     f"{resource_registration_domain}/processes/deregister/execution",
#     json=json_dict
# )
# response.raise_for_status()
# response

<Response [200]>

## Verify Registration Job Status

In [9]:
jobs_url = f"{resource_registration_domain}/jobs"
response = requests.get(jobs_url)
jobs = response.json()

print(f"Jobs listing status code: {response.status_code}")
latest_job = jobs["jobs"][0] if jobs.get("jobs") else {}
print(json.dumps(latest_job, indent=2))

Jobs listing status code: 200
{
  "type": "process",
  "processID": "register",
  "jobID": "b6fdebdc-16c5-11f0-b356-529988b485c6",
  "status": "successful",
  "message": "Job complete",
  "progress": 100,
  "parameters": null,
  "job_start_datetime": "2025-04-11T11:11:28.294894Z",
  "job_end_datetime": "2025-04-11T11:11:29.636425Z",
  "links": [
    {
      "href": "https://registration-api.notebook-test.develop.eoepca.org/jobs/b6fdebdc-16c5-11f0-b356-529988b485c6/results?f=html",
      "rel": "http://www.opengis.net/def/rel/ogc/1.0/results",
      "type": "text/html",
      "title": "Results of job as HTML"
    },
    {
      "href": "https://registration-api.notebook-test.develop.eoepca.org/jobs/b6fdebdc-16c5-11f0-b356-529988b485c6/results?f=json",
      "rel": "http://www.opengis.net/def/rel/ogc/1.0/results",
      "type": "application/json",
      "title": "Results of job as JSON"
    }
  ]
}


## Validate Registered Dataset in Resource Discovery

In [ ]:
resource_discovery_domain = f'{os.environ.get("HTTP_SCHEME")}://resource-catalogue.{platform_domain}'
metadata_items_url = f"{resource_discovery_domain}/collections/metadata:main/items"

response = requests.get(metadata_items_url)
items = response.json().get("features", [])

item_ids = [item.get("id") for item in items]
expected_item_id = stac_item_json_data.get("id")

if expected_item_id in item_ids:
    print(f"✅ Dataset '{expected_item_id}' successfully registered and discoverable!")
    print(f"View it here: {metadata_items_url}/{expected_item_id}")
else:
    print(f"❌ Dataset '{expected_item_id}' registration failed or item not discoverable.")

✅ Dataset 'S2B_MSIL2A_20190910T095029_N0213_R079_T33UXP_20190910T124513.SAFE' successfully registered and discoverable!
Check the link here: https://resource-catalogue.notebook-test.develop.eoepca.org/collections/metadata:main/items/S2B_MSIL2A_20190910T095029_N0213_R079_T33UXP_20190910T124513.SAFE


## Detailed Inspection of `hello-world` Process

In [12]:
hello_world_process_url = f"{resource_registration_domain}/processes/hello-world"
response = requests.get(hello_world_process_url)
print(json.dumps(response.json(), indent=2)[:500])

{
  "version": "0.2.0",
  "id": "hello-world",
  "title": "Hello World",
  "description": "An example process that takes a name as input, and echoes it back as output. Intended to demonstrate a simple process with a single literal input.",
  "jobControlOptions": [
    "sync-execute",
    "async-execute"
  ],
  "keywords": [
    "hello world",
    "example",
    "echo"
  ],
  "links": [
    {
      "type": "text/html",
      "rel": "about",
      "title": "information",
      "href": "https://exa


## Synchronous Execution of Hello-World Process

In [13]:
hello_world_exec_url = f"{hello_world_process_url}/execution"
payload = {
    "inputs": {
        "name": "EOEPCA User",
        "message": "Testing synchronous execution"
    }
}
response = requests.post(hello_world_exec_url, json=payload)
print(response.json())

{'id': 'echo', 'value': 'Hello EOEPCA User! Testing synchronous execution'}


## Asynchronous Execution of Hello-World Process

In [14]:
headers = {"Prefer": "respond-async"}
payload = {
    "inputs": {
        "name": "EOEPCA User",
        "message": "Testing asynchronous execution"
    }
}
response = requests.post(hello_world_exec_url, json=payload, headers=headers)
job_location = response.headers.get('Location')
print(f"Job created asynchronously. Location: {job_location}")

Job created asynchronously. Location: https://registration-api.notebook-test.develop.eoepca.org/jobs/ddd5b1f4-16c5-11f0-9ee2-529988b485c6


## Check Status of Async Job

In [15]:
response = requests.get(job_location)
print(json.dumps(response.json(), indent=2))

{
  "type": "process",
  "processID": "hello-world",
  "jobID": "ddd5b1f4-16c5-11f0-9ee2-529988b485c6",
  "status": "successful",
  "message": "Job complete",
  "progress": 100,
  "parameters": null,
  "job_start_datetime": "2025-04-11T11:12:33.462430Z",
  "job_end_datetime": "2025-04-11T11:12:33.473194Z",
  "links": [
    {
      "href": "https://registration-api.notebook-test.develop.eoepca.org/jobs/ddd5b1f4-16c5-11f0-9ee2-529988b485c6/results?f=html",
      "rel": "http://www.opengis.net/def/rel/ogc/1.0/results",
      "type": "text/html",
      "title": "Results of job as HTML"
    },
    {
      "href": "https://registration-api.notebook-test.develop.eoepca.org/jobs/ddd5b1f4-16c5-11f0-9ee2-529988b485c6/results?f=json",
      "rel": "http://www.opengis.net/def/rel/ogc/1.0/results",
      "type": "application/json",
      "title": "Results of job as JSON"
    }
  ]
}


## Retrieve Results from Async Job

In [ ]:
job_results_url = f"{job_location}/results"
response = requests.get(job_results_url, headers={"Accept": "application/json"})
print(response.json())

{'id': 'echo', 'value': 'Hello EOEPCA User! Testing asynchronous execution'}


# Running Workflows

In [74]:
flowable_domain = f'{os.environ.get("HTTP_SCHEME")}://registration-harvester-api.{platform_domain}'
flowable_docs_url = f"{flowable_domain}/flowable-rest/docs"
response = requests.get(flowable_docs_url)

print(f"Flowable Swagger UI status code: {response.status_code}")
response.url

Flowable Swagger UI status code: 200


'https://registration-harvester-api.notebook-test.develop.eoepca.org/flowable-rest/docs/'

In [75]:
from requests import Session
from requests.auth import HTTPBasicAuth
import json
import requests
import os

flowable_admin_user = os.environ.get("FLOWABLE_ADMIN_USER", "eoepca")
flowable_admin_password = os.environ.get("FLOWABLE_ADMIN_PASSWORD", "eoepca")
flowable_session = Session()
flowable_session.auth = HTTPBasicAuth(flowable_admin_user, flowable_admin_password)

## List Flowable Deployments

In [76]:
deployments_url = f"{flowable_domain}/flowable-rest/service/repository/deployments"
response = flowable_session.get(deployments_url)
deployments = response.json().get("data", [])

print(f"Found {len(deployments)} deployments.")

if deployments:
    latest_deployment = deployments[0]
    deployment_id = latest_deployment.get("id")
    for idx, deployment in enumerate(deployments, 1):
        print("%-2s %-25s deployed at: %-30s with id: %s" % (idx, deployment['name'], deployment['deploymentTime'], deployment["id"]))

Found 2 deployments.
1  Demo processes            deployed at: 2025-03-21T15:51:44.230Z       with id: 636540d0-066c-11f0-9621-dac2883c12fb
2  landsat                   deployed at: 2025-04-09T12:05:16.406Z       with id: e6451a22-153a-11f0-9621-dac2883c12fb


## Deploy Landsat Workflow

In [77]:
landsat_bpmn_files = [
    "https://raw.githubusercontent.com/EOEPCA/registration-harvester/refs/heads/main/workflows/landsat.bpmn", 
    "https://raw.githubusercontent.com/EOEPCA/registration-harvester/refs/heads/main/workflows/landsat-scene-ingestion.bpmn"
]

for bpmn in landsat_bpmn_files:
    bpmn_file = {os.path.basename(bpmn): requests.get(bpmn).text}
    response = flowable_session.post(deployments_url, files=bpmn_file)
    print(f"Sucessfully deployed workflow: {bpmn_file} with status code: {response.status_code}")

Sucessfully deployed workflow: {'landsat.bpmn': '<?xml version="1.0" encoding="UTF-8"?>\n<definitions xmlns="http://www.omg.org/spec/BPMN/20100524/MODEL" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:bpmndi="http://www.omg.org/spec/BPMN/20100524/DI" xmlns:omgdc="http://www.omg.org/spec/DD/20100524/DC" xmlns:omgdi="http://www.omg.org/spec/DD/20100524/DI" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:flowable="http://flowable.org/bpmn" xmlns:design="http://flowable.org/design" targetNamespace="http://flowable.org/test" design:palette="flowable-work-process-palette">\n  <collaboration id="Collaboration">\n    <participant id="Pool_1" name="Landsat Registration" processRef="landsatRegistration" />\n  </collaboration>\n  <process id="landsatRegistration" name="Landsat Workflow" isExecutable="true" flowable:candidateStarterGroups="flowableUser">\n    <extensionElements>\n      <design:stencilid>BPMNDiagram</design:stencilid>\n      <design:creationdate>2024-12-02T11:03:29.

## List Flowable Deployed Processes

In [78]:
process_definitions_url = f"{flowable_domain}/flowable-rest/service/repository/process-definitions"
print(f"Flowable process definitions URL: {process_definitions_url}")

response = flowable_session.get(process_definitions_url)
processes = response.json()["data"]

for idx, process in enumerate(processes, 1):
    print("%-2s %-28s version: %-5s id: %s" % (idx, process['name'], process['version'], process['id']))
    if process["name"] == "Landsat Workflow":
        landsat_process_id = process["id"]

Flowable process definitions URL: https://registration-harvester-api.notebook-test.develop.eoepca.org/flowable-rest/service/repository/process-definitions
1  Create timers process        version: 1     id: createTimersProcess:1:641b2360-066c-11f0-9621-dac2883c12fb
2  Famous One Task Process      version: 1     id: oneTaskProcess:1:641b235f-066c-11f0-9621-dac2883c12fb
3  Fix system failure           version: 1     id: fixSystemFailure:1:641b235c-066c-11f0-9621-dac2883c12fb
4  Helpdesk process             version: 1     id: escalationExample:1:641b235e-066c-11f0-9621-dac2883c12fb
5  Landsat Scene Ingestion      version: 1     id: landsatSceneIngestion:1:03aaa058-153c-11f0-9621-dac2883c12fb
6  Landsat Workflow             version: 1     id: landsatRegistration:1:e656a655-153a-11f0-9621-dac2883c12fb
7  Landsat Workflow             version: 2     id: landsatRegistration:2:037bef31-153c-11f0-9621-dac2883c12fb
8  Review sales lead            version: 1     id: reviewSaledLead:1:641b235d-066c-

## Execute Landsat Workflow

In [81]:
query = json.dumps({ "created": { "gte": "2024-12-13T15:00:00.000000Z", "lt": "2024-12-13T16:00:00.000000Z" } })
variables = [
    {"name": "datetime_interval", "type": "string", "value": "2024-11-12T15:00:00.000000Z/2024-11-12T16:00:00.000000Z"},    
    {"name": "collections", "type": "string", "value": "landsat-c2l2-sr"},
    {"name": "bbox", "type": "string", "value": "8,40,18,60"},
    {"name": "query", "type": "string", "value": query},
]

# Create the JSON body for HTTP request which triggers the workflow
body = {}
body["processDefinitionId"] = landsat_process_id
body["variables"] = variables

response = flowable_session.post(process_instances_url, json=body)
print(f"POST {process_instances_url}")
print(response.text)


POST https://registration-harvester-api.notebook-test.develop.eoepca.org/flowable-rest/service/runtime/process-instances
{"id":"1f3603aa-153c-11f0-9621-dac2883c12fb","url":"https://registration-harvester-api.notebook-test.develop.eoepca.org/flowable-rest/service/runtime/process-instances/1f3603aa-153c-11f0-9621-dac2883c12fb","name":null,"businessKey":null,"businessStatus":null,"suspended":false,"ended":false,"processDefinitionId":"landsatRegistration:2:037bef31-153c-11f0-9621-dac2883c12fb","processDefinitionUrl":"https://registration-harvester-api.notebook-test.develop.eoepca.org/flowable-rest/service/repository/process-definitions/landsatRegistration:2:037bef31-153c-11f0-9621-dac2883c12fb","processDefinitionName":"Landsat Workflow","processDefinitionDescription":null,"activityId":null,"startUserId":"eoepca","startTime":"2025-04-09T12:14:01.434Z","superProcessInstanceId":null,"variables":[{"name":"collections","type":"string","value":"landsat-c2l2-sr","scope":"local"},{"name":"initiato

## Check Workflows

In [82]:
process_instances_url = f"{flowable_domain}/flowable-rest/service/runtime/process-instances"
print(f"GET {process_instances_url}")
response = flowable_session.get(process_instances_url)
print(response.text)
process_instances = response.json()["data"]
if len(process_instances) == 0:
    print("No running workflows")
else:
    for idx, process in enumerate(process_instances, 1):
        print("%s %-25s started at: %-25s id: %s" % (idx, process['processDefinitionName'], process['startTime'], process['id']))

GET https://registration-harvester-api.notebook-test.develop.eoepca.org/flowable-rest/service/runtime/process-instances
{"data":[{"id":"0935155a-153c-11f0-9621-dac2883c12fb","url":"https://registration-harvester-api.notebook-test.develop.eoepca.org/flowable-rest/service/runtime/process-instances/0935155a-153c-11f0-9621-dac2883c12fb","name":null,"businessKey":null,"businessStatus":null,"suspended":false,"ended":false,"processDefinitionId":"landsatRegistration:2:037bef31-153c-11f0-9621-dac2883c12fb","processDefinitionUrl":"https://registration-harvester-api.notebook-test.develop.eoepca.org/flowable-rest/service/repository/process-definitions/landsatRegistration:2:037bef31-153c-11f0-9621-dac2883c12fb","processDefinitionName":"Landsat Workflow","processDefinitionDescription":null,"activityId":null,"startUserId":"eoepca","startTime":"2025-04-09T12:13:24.518Z","superProcessInstanceId":null,"variables":[],"callbackId":null,"callbackType":null,"referenceId":null,"referenceType":null,"propagate

## Delete Workflows

In [72]:
response = flowable_session.get(deployments_url)
deployments = response.json().get("data", [])
for idx, deployment in enumerate(deployments, 1):
    print(f'Delete: {deployments_url}/{deployment["id"]}')
    flowable_session.delete(f"{deployments_url}/{deployment['id']}")

Delete: https://registration-harvester-api.notebook-test.develop.eoepca.org/flowable-rest/service/repository/deployments/636540d0-066c-11f0-9621-dac2883c12fb
Delete: https://registration-harvester-api.notebook-test.develop.eoepca.org/flowable-rest/service/repository/deployments/e6451a22-153a-11f0-9621-dac2883c12fb
